### Importaciones.

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import random
import gym

### Definir la clase para el agente de DQN.

In [8]:
class DQNAgent:
    # * constructor de la clase
    def __init__(self, state_size, action_size):
        # * propiedades de la clase
        
        # * la cantidad de entradas y salidas que tendra el modelo
        # ? numero de entradas (caracteristicas del entorno,velocidad, posición)
        self.state_size = state_size
        # ? numero de acciones posibles (izquierda o derecha)
        self.action_size = action_size

        # ? red neuronal principal que hara las predicciones de valores de Q
        self.model = self._build_model()
        # ? copia congelada de la red que se actualiza periodicamente. Evita las inestabilidades
        self.target_model = self._build_model()
        # ? actualiza la copia del modelo (copia los pesos de model a target_model)
        self.update_target_model()

        # ? memoria del modelo donde se guardaran sus experiencias pasadas.
        self.memory = []  
        self.gamma = 0.95 # ? el valor que le da el agente a las recompensas futuras (entre 0 y 1)
        self.epsilon = 1.0 # ? controla la exploración vs explotación (cuánto se arriesga a probar cosas nuevas).
        self.epsilon_decay = 0.995 # ? cómo disminuye epsilon en cada episodio para que el agente explore menos y explote más.
        self.epsilon_min = 0.01 # ? mínimo valor de epsilon

    # * metodo para construir el modelo (red neuronal)
    # ? tendra 2 capas ocultas con 24 neuronas
    # ? capa final tiene tantas salidas como acciones posibles
    # ? utiliza el optimizador "Adam" usando MSE (error cuadratico medio)
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(optimizer='adam', loss='mse')
        return model

    # * elegir que accion se realizara
    def act(self, state):
        # ? si rand es menor o igual a epsilon actua al hacer
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size) # ? realiza una accion aleatoria.
        q_values = self.model.predict(state[np.newaxis], verbose=0) # ? realiza una prediccion.
        return np.argmax(q_values[0]) # ? accion con mayor Q (exploracion)

    # * guarda las experiencias vividas por el agente para luego usarlas en el entrenamiento.
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    # * toma un conjunto aleatorio de experiencias de la memoria
    def replay(self, batch_size=32):
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        
        
        for state, action, reward, next_state, done in minibatch:
            # ? TARGET contiene las predicciones actuales de Q
            target = self.model.predict(state[np.newaxis], verbose=0)
            if done:
                target[0][action] = reward # ? si termino el juego, no hay recompensas a futuro.
            else:
                # ? si no termino se aplica la ecuacion de Bellman.
                t = self.target_model.predict(next_state[np.newaxis], verbose=0)
                target[0][action] = reward + self.gamma * np.amax(t[0])
            # ? Ajusta la red para que prediga el valor Q actualizado.
            self.model.fit(state[np.newaxis], target, epochs=1, verbose=0)

        # ? Reduce epsilon gradualmente para que el agente explore menos con el tiempo
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    # ? Copia los pesos del modelo principal al modelo objetivo.
    # ? Se hace periódicamente (cada ciertos episodios) para mejorar la estabilidad del entrenamiento.
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())


### Juego de entrenamiento.

In [10]:
env = gym.make("CartPole-v1")
agent = DQNAgent(state_size=4, action_size=2)

for e in range(5):
    print(f"Intento N° {e}")
    state, _ = env.reset()
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        agent.replay()
    agent.update_target_model()


c:\Users\IPF-2025\Desktop\MATERIALES\py-ciencia-datos-practica\keras-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Intento N° 0
Intento N° 1
Intento N° 2
Intento N° 3
Intento N° 4


### Explicaciones:

#### GYM

* **Gym**: (ahora conocido como Gymnasium, anteriormente mantenido por OpenAI) es una librería de Python para simular entornos de **aprendizaje por refuerzo** (RL). Se podria decir que es una **"sala de entrenamiento"** para agentes.

* Permite
1. Probar algoritmos de RL fácilmente.
2. Usar entornos listos para entrenar (juegos, robótica, navegación, etc).
3. Medir desempeño y compararlo entre algoritmos.

#### CARTPOLE

* Es uno de los entornos más clásicos de RL, incluido en gym:
* ¿Como funciona el juego?

Hay un carrito sobre un riel, con un palo vertical (como un péndulo invertido).

El agente puede mover el carrito izquierda o derecha.

El objetivo es mantener el palo en equilibrio vertical el mayor tiempo posible.

En resumidas cuentas, es un entorno donde un agente debe equilibrar un palo vertical moviendo un carrito.


#### Agente:

* **model**: Red neuronal principal que predice los valores Q.
* **target_model**: Copia estable usada para calcular valores objetivo.
* **memory**: Historial de experiencias para entrenar de forma más estable
* **act**: Decide si explorar o explotar una acción
* **replay**: Entrena la red usando experiencias guardadas
* **epsilon**: Controla cuánto se explora vs se explota, probabilidad de explorar (vs explotar) en cada paso. Representa cuanta curiosidad tiene el agente.
* **decaimiento**: Epsilon se reduce para que el agente explore menos a medida que aprende

* **exploración**: Prueba nuevas acciones para aprender más del entorno
(Quiere probar cosas nuevas (aunque no sepa si son buenas) → eso es explorar.)
* **explotación**: Usa el conocimiento actual para elegir la mejor acción conocida
(Otras veces prefiere hacer lo que ya sabe que funciona → eso es explotar.)

##### Importancia de exploracion y explotacion:
* Si el agente solo explota, puede quedarse atascado en una solución subóptima (no descubre opciones mejores).
* Si el agente solo explora, nunca usa lo aprendido para mejorar sus resultados.

##### Ejemplos de Epsilon:
* Si **epsilon = 1.0** el robot hace acciones aleatorias todo el tiempo (explora 100%).
*Si **epsilon = 0.0** el robot nunca prueba cosas nuevas, solo hace lo que ya aprendió (explotación total).

#### **Q** (quality):
Es una estimación de lo bueno que es hacer una acción en una situación.

##### Ejemplo del funcionamiento de Q:

1. Estado: el palo está inclinado hacia la derecha.

2. El robot puede hacer 2 cosas:
* Acción 0: mover a la izquierda.
* Acción 1: mover a la derecha.

3. El modelo estima algo asi:
* Q(estado, izquierda) = 0.5
* Q(estado, derecha) = 0.9
Esto significa que el modelo piensa, "Si hago 'derecha' ahora, probablemente me irá mejor".

#### Ecuacion de Bellman:
Se trata del corazon del aprendizaje por refuerzo, se utiliza para que el agente aprenda a tomar mejores decisiones.

"El valor de estar en un estado y tomar una acción es igual a la recompensa inmediata que obtienes más el valor futuro esperado que obtendrás después".

* Valor presente = recompensa de ahora + valor del futuro

##### Utilidades:
1. Aprender valores Q de manera inteligente.
2. Actualizar el conocimiento en cada paso del agente.
3. Guiar las decisiones futuras del agente hacia mejores recompensas.

